# Research Environment Setup Guide

This notebook provides an interactive guide to setting up and configuring the research environment.

## 1. Environment Information

First, let's check the current environment configuration and system resources.

In [ ]:
import sys
import os
import platform
import subprocess
import psutil

# System information
print(f"Python version: {platform.python_version()}")
print(f"Platform: {platform.platform()}")
print(f"CPU cores: {os.cpu_count()}")
print(f"Memory: {psutil.virtual_memory().total / (1024**3):.1f} GB")

# Check for GPU
def check_gpu():
    try:
        output = subprocess.check_output(['nvidia-smi'], stderr=subprocess.STDOUT).decode('utf-8')
        return f"GPU detected:\n{output.splitlines()[0]}\n{output.splitlines()[1]}"
    except (subprocess.SubprocessError, FileNotFoundError):
        return "No NVIDIA GPU detected or drivers not installed"

print(check_gpu())

## 2. Required Dependencies

The following packages are required for the research environment. Let's check if they're installed.

In [ ]:
# Check installed packages
def check_package(package_name):
    try:
        __import__(package_name)
        return True
    except ImportError:
        return False

essential_packages = [
    'numpy', 'pandas', 'matplotlib', 'scipy', 'sklearn',
    'torch', 'tensorflow', 'jupyter', 'plotly'
]

print("Package Status:")
for package in essential_packages:
    status = "✓ Installed" if check_package(package) else "✗ Not installed"
    print(f"{package}: {status}")

### Install Missing Packages

You can run the cell below to install any missing packages.

In [ ]:
# Install missing packages
def install_missing_packages():
    missing_packages = [pkg for pkg in essential_packages if not check_package(pkg)]
    
    if not missing_packages:
        print("All essential packages are already installed.")
        return
    
    print(f"Installing missing packages: {', '.join(missing_packages)}")
    
    for package in missing_packages:
        print(f"Installing {package}...")
        try:
            # Use a subprocess to avoid affecting the current Python process
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            print(f"{package} installed successfully.")
        except subprocess.CalledProcessError:
            print(f"Failed to install {package}. Please install it manually.")
    
    print("\nPackage installation complete. You may need to restart the kernel.")
    print("Use the 'Kernel > Restart Kernel' menu option to restart.")

# Uncomment to run
# install_missing_packages()

## 3. Docker Configuration

Check if Docker is installed and properly configured.

In [ ]:
# Check Docker installation
def check_docker():
    try:
        output = subprocess.check_output(['docker', '--version'], stderr=subprocess.STDOUT).decode('utf-8')
        docker_compose_output = subprocess.check_output(['docker-compose', '--version'], stderr=subprocess.STDOUT).decode('utf-8')
        return f"Docker: {output.strip()}\nDocker Compose: {docker_compose_output.strip()}"
    except (subprocess.SubprocessError, FileNotFoundError):
        return "Docker not found. Please install Docker and Docker Compose."

# Check NVIDIA Docker
def check_nvidia_docker():
    try:
        output = subprocess.check_output(['docker', 'info'], stderr=subprocess.STDOUT).decode('utf-8')
        if 'nvidia' in output:
            return "NVIDIA Docker runtime is available."
        else:
            return "NVIDIA Docker runtime not detected. GPU containers may not work properly."
    except (subprocess.SubprocessError, FileNotFoundError):
        return "Unable to check Docker configuration."

print(check_docker())
print("\n" + check_nvidia_docker())

### Docker Setup Instructions

If Docker is not installed or properly configured, follow these instructions:

#### Installing Docker

**For Windows:**
1. Download and install Docker Desktop from [https://www.docker.com/products/docker-desktop](https://www.docker.com/products/docker-desktop)
2. Follow the installation instructions
3. Start Docker Desktop from the Start menu

**For macOS:**
1. Download and install Docker Desktop from [https://www.docker.com/products/docker-desktop](https://www.docker.com/products/docker-desktop)
2. Follow the installation instructions
3. Start Docker Desktop from the Applications folder

**For Linux (Ubuntu):**
```bash
# Update package index
sudo apt-get update

# Install prerequisites
sudo apt-get install apt-transport-https ca-certificates curl software-properties-common

# Add Docker's official GPG key
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

# Add Docker repository
sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable"

# Update package index again
sudo apt-get update

# Install Docker
sudo apt-get install docker-ce docker-compose

# Add your user to the docker group
sudo usermod -aG docker $USER
```

#### Setting up NVIDIA Docker (for GPU support)

```bash
# Add the NVIDIA Container Toolkit package repositories
distribution=$(. /etc/os-release;echo $ID$VERSION_ID)
curl -s -L https://nvidia.github.io/nvidia-docker/gpgkey | sudo apt-key add -
curl -s -L https://nvidia.github.io/nvidia-docker/$distribution/nvidia-docker.list | sudo tee /etc/apt/sources.list.d/nvidia-docker.list

# Update the package listing
sudo apt-get update

# Install nvidia-docker2 package
sudo apt-get install -y nvidia-docker2

# Restart Docker service
sudo systemctl restart docker
```

## 4. GPU Setup and Configuration

Run the GPU setup script to configure the environment for optimal GPU usage.

In [ ]:
# Run GPU setup script
def run_gpu_setup():
    script_path = os.path.join('..', 'setup', 'gpu_setup.sh')
    
    if not os.path.exists(script_path):
        print(f"GPU setup script not found at {script_path}")
        return
    
    if platform.system() == "Windows":
        print("For Windows, please run the GPU setup script manually from a command prompt:")
        print("1. Open a command prompt with administrator privileges")
        print("2. Navigate to the setup directory")
        print("3. Run: bash gpu_setup.sh")
        return
    
    print("Running GPU setup script. This may take a few minutes...")
    try:
        # Make script executable
        subprocess.check_call(['chmod', '+x', script_path])
        
        # Run script
        output = subprocess.check_output(['bash', script_path], stderr=subprocess.STDOUT).decode('utf-8')
        print(output)
        print("GPU setup completed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"GPU setup failed with error code {e.returncode}:")
        print(e.output.decode('utf-8'))
        print("\nYou may need to run the script manually with sudo privileges.")

# Uncomment to run
# run_gpu_setup()

## 5. Test Environment

Now let's test the environment to ensure everything is working properly.

In [ ]:
# Import system utilities for testing
sys.path.append('..')
try:
    from utils.system_utils import system_manager
    from utils.gpu_utils import gpu_manager
    
    # System resources
    system_summary = system_manager.get_system_summary()
    print("System Resources:")
    print(f"CPU Usage: {system_summary['cpu_percent']:.1f}%")
    print(f"Memory Usage: {system_summary['memory_percent']:.1f}%")
    print(f"Disk Usage: {system_summary['disk_percent']:.1f}%")
    
    # GPU resources
    gpu_available = gpu_manager.check_gpu_availability()
    print(f"\nGPU Available: {gpu_available}")
    
    if gpu_available:
        gpu_info = gpu_manager.get_gpu_info()
        for idx, gpu in enumerate(gpu_info):
            print(f"GPU {idx}: {gpu.get('name', 'Unknown')}")
            print(f"  Memory: {gpu.get('memory_used_mb', 0)} MB / {gpu.get('memory_total_mb', 0)} MB")
            print(f"  Utilization: {gpu.get('utilization_percent', 0)}%")
    
    print("\nEnvironment test completed successfully!")
except ImportError as e:
    print(f"Import error: {e}")
    print("The utility modules may not be properly installed.")
    print("Make sure you've run the setup scripts in the setup directory.")
except Exception as e:
    print(f"Error while testing environment: {e}")

## 6. Next Steps

Now that your environment is set up, here are some next steps you can take:

1. **Explore system diagnostics**: Check the `system_diagnostics.ipynb` notebook for detailed system monitoring.

2. **Test GPU performance**: Run the `gpu_performance_testing.ipynb` notebook to benchmark your GPU performance.

3. **Work with Docker containers**: Use the `environment_manager.py` script to start containerized environments:
   ```bash
   python environment_manager.py --enable-monitoring
   ```

4. **Configure monitoring**: Set up monitoring dashboards by enabling the monitoring options:
   ```bash
   python environment_manager.py --enable-monitoring --monitor-port 3000
   ```

5. **Add custom frameworks**: Extend the environment with additional frameworks by modifying the Docker configurations in the `docker` directory.

6. **Create your own experiments**: Use the `notebooks` directory to create and organize your research experiments.